In [1]:
import cv2
import numpy as np
import serial
import time

cap = cv2.VideoCapture(1) 
if not cap.isOpened():
    raise IOError("Cannot open webcam")

def nothing(x):
    pass

cv2.namedWindow("HSV Slider Tool", cv2.WINDOW_NORMAL)
cv2.resizeWindow("HSV Slider Tool", 400, 300)
cv2.createTrackbar("LH", "HSV Slider Tool", 25, 179, nothing)
cv2.createTrackbar("LS", "HSV Slider Tool", 30, 255, nothing)
cv2.createTrackbar("LV", "HSV Slider Tool", 50, 255, nothing)
cv2.createTrackbar("UH", "HSV Slider Tool", 35, 179, nothing)
cv2.createTrackbar("US", "HSV Slider Tool", 60, 255, nothing)
cv2.createTrackbar("UV", "HSV Slider Tool", 90, 255, nothing) 
try:
    arduino = serial.Serial('COM5', 115200) 
    time.sleep(2) 
except serial.SerialException as e:
    print(f"Error opening serial port: {e}")
    arduino = None

def send_command(direction, step):
    if arduino and arduino.is_open:
        cmd = f"{direction}:{step}"
        arduino.write((cmd + "\n").encode())

last_direction = ""

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    height, width, _ = frame.shape
    center_x = width // 2

    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    lh = cv2.getTrackbarPos("LH", "HSV Slider Tool")
    ls = cv2.getTrackbarPos("LS", "HSV Slider Tool")
    lv = cv2.getTrackbarPos("LV", "HSV Slider Tool")
    uh = cv2.getTrackbarPos("UH", "HSV Slider Tool")
    us = cv2.getTrackbarPos("US", "HSV Slider Tool")
    uv = cv2.getTrackbarPos("UV", "HSV Slider Tool") 
    lower = np.array([lh, ls, lv])
    upper = np.array([uh, us, uv])

    mask = cv2.inRange(hsv, lower, upper)
    result = cv2.bitwise_and(frame, frame, mask=mask)

    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    direction = "CENTER"
    step = 0

    if contours:
        c = max(contours, key=cv2.contourArea)
        area = cv2.contourArea(c)

        if area > 1000: 
            x, y, w, h = cv2.boundingRect(c)
            cx = x + w // 2
            
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.circle(frame, (cx, y + h // 2), 5, (255, 0, 0), -1)
            cv2.line(frame, (center_x, 0), (center_x, height), (0, 255, 255), 2)
            cv2.line(frame, (cx, 0), (cx, height), (0, 255, 255), 2) 
            
            error = cx - center_x
            deadband = 20 
            
            if error > deadband:
                direction = "RIGHT"
                step = int(abs(error) * 0.05) 
                step = min(step, 10) 
            elif error < -deadband:
                direction = "LEFT"
                step = int(abs(error) * 0.05)
                step = min(step, 10)
            else:
                direction = "CENTER"
                step = 0
    
    cv2.putText(frame, f"Direction: {direction}", (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
    cv2.putText(frame, f"Step: {step}", (20, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)

    if direction != last_direction or step > 0:
        send_command(direction, step)
        print(f"Command Sent: {direction} with step {step}")
        last_direction = direction

    cv2.imshow("Webcam Feed", frame)
    cv2.imshow("Mask", mask)
    cv2.imshow("Result", result)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Error opening serial port: could not open port 'COM5': FileNotFoundError(2, 'The system cannot find the file specified.', None, 2)
Command Sent: CENTER with step 0


KeyboardInterrupt: 

In [8]:
!pip install opencv-python numpy pyserial


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip
